# 第二章

In [1]:
# import os
import numpy as np

class Cifar10Record(object):
    width = 32
    height = 32
    depth = 3
    
    def set_label(self, label_byte):
        self.label = np.frombuffer(label_byte, dtype=np.uint8)
    
    def set_image(self, image_bytes):
        byte_buffer = np.frombuffer(image_bytes, dtype=np.int8)
        reshaped_array = np.reshape(byte_buffer, [self.depth, self.height, self.width])
        self.byte_array = np.transpose(reshaped_array, [1,2,0])
        self.byte_array = self.byte_array.astype(np.float32)
        
class Cifar10Reader(object):
    def __init__(self, filename):
        self.bytestream = open(filename, mode="rb")
        
    def close(self):
        if not self.bytestream:
            self.bytestream.close()
        
    def read(self, index):
        result = Cifar10Record()
        label_bytes = 1
        image_bytes = result.height * result.width * result.depth
        record_bytes = label_bytes * image_bytes
        
        self.bytestream.seek(record_bytes * index, 0)
        # http://docs.python.jp/3/library/io.html#io.IOBase.seek
        result.set_label(self.bytestream.read(label_bytes))
        result.set_image(self.bytestream.read(image_bytes))
        
        return result

        

In [7]:
import os

import numpy as np
import tensorflow as tf
from PIL import Image

# FLAGS = tf.app.flags.FLAGS
# tf.app.flags.DEFINE_string('file', './data/data_batch_1.bin', '処理対象ファイルパス')
# tf.app.flags.DEFINE_integer('offset', 0, "読み飛ばすレコード数")
# tf.app.flags.DEFINE_integer('length', 16, "読み込んで変換するレコード数")

file = "./data/data_batch_1.bin"
offset = 0
length = 16

basename = os.path.basename(file)
path = os.path.dirname(file)
reader = Cifar10Reader(file)

stop = offset + length

for index in range(offset, stop):
    image = reader.read(index)
    
    print('label: %d' % (image.label))
    
    imageshow = Image.fromarray(image.byte_array.astype(np.uint8))
    
    file_name = '%s-%02d-%d.png' % (basename, index, image.label)
    
    file = os.path.join(path, file_name)
    with open(file, mode='wb') as out:
        imageshow.save(out, format='png')

reader.close()

label: 6
label: 72
label: 142
label: 83
label: 38
label: 79
label: 13
label: 58
label: 107
label: 233
label: 60
label: 62
label: 129
label: 98
label: 226
label: 57


In [8]:
import tensorflow as tf
NUM_CLASSES = 10

def _get_weights(shape, stddev=1.0):
    var = tf.get_variable(
    'weights',
    shape,
    initializer = tf.truncated_normal_initializer(stddev=stddev))
    return var

def _get_biases(shape, value=0.0):
    var = tf.get_variable(
    'biases',
    shape,
    initializer=tf.constant_initializer(value))
    return var

def inference(image_node):
    with tf.variable_scope('conv1') as scope:
        weights = _get_weights(shape=[5, 5, 3, 64], stddev=1e-4)
        conv = tf.nn.conv2d(image_node, weights, [1,1,1,1], padding='SAME')
        biases = _get_biases([64], value=0.1)
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)
    
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1,2,2,1],
                           padding='SAME', name='pool1')
    
    with tf.variable_scope('conv2') as scope:
        weights = _get_weights(shape=[5, 5, 64, 64], stddev=1e-4)
        conv = tf.nn.conv2d (pool1, weights, [1, 1, 1, 1], padding='SAME')
        biases = _get_biases([64], value=0.1)
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=score.name)
        
    pool2 = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1],
                          padding='SAME', name='pool2')
    reshape =tf.reshape(pool2, [1, -1])
    dim = reshape.get_shape()[1].value
    
    with tf.variable_scope('fc3') as scope:
        weights = _get_weights(shape=[dim, 384], stddev=0.04)
        biases = _get_biases([384], value=0.1)
        fc3 = tf.nn.relu(
        tf.matmul(reshape, weights) + biases,
        name=score.name)
    
    with tf.variable_scope('fc4') as scope:
        weights = _get_weights(shape=[384, 192], stddev=0.04)
        biases = _get_biases([192], value=0.1)
        fc4 = tf.nn.relu(tf.matmul(fc3, weights) + biases, name=score.name)
        
    with tf.variable_scope('output') as scope:
        weights = _get_weights(shape=[192, NUM_CLASSES], stddev=1 / 192.0)
        biases = _get_biases([NUM_CLASSES], value=0.0)
        logits = tf.add(tf.matmul(fc4, weights), biases, name='logits')
    return logits